Name: Shiska Raut <br>
ID: 1001526329

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline

## Read training/evaluation data

**Argument(s):** name of a .txt file with each line containing training/evaluation features(x) and label(y) in the following format:
((x1, x2, .....xn), y)  

**Return(s):** 'X, Y' where X is a numpy array of feature vectors and Y is the target label vector.
Note: Each column in the array(s) epresents a single datapoint.

In [33]:
# given a file
def get_X_Y_arrays(filename):
    try:
        f = open(filename, 'r')
    except OSError:
        print(f'{filename} could not be opened.\n')
        sys.exit()
        
    # initialize list to store feature and labels for training data
    features = []             
    labels = []
    
    with f:
        line = f.readline()
        while line != '':
            # strip newline and outer parenthesis
            line = line.strip('\n')
            line = line.strip('( )')
            
            # extrace label and append to labels list
            single_label = line.split('), ')[-1]
            labels.append(single_label)
            
            # extrace features and append to features list
            feat = line.split('), ')[0].split(', ')
            features.append(feat)
            
            # read next line
            line = f.readline()
        
        # create dataframe of features and append labels
        X = np.array(features, dtype = float, ndmin = 2)
        
        # convert labels list to array
        Y = np.array(labels, dtype = str, ndmin = 2)
        
        return X.transpose(), Y
    

## Read test data

**Argument(s):** name of a .txt file with each line containing test features(x) in the following format:
(x1, x2, .....xn)

**Return(s):** 'X' where X is a numpy array of feature vectors.
Note: Each column in the array(s) epresents a single datapoint.

In [10]:
def get_X_array(filename):
    try:
        f = open(filename, 'r')
    except OSError:
        print(f'{filename} could not be opened.\n')
        sys.exit()
        
    # initialize list to store feature and labels for training data
    features = []             
    
    with f:
        line = f.readline()
        while line != '':
            
            # get feature values
            line = line.strip('\n')
            line = line.strip('( )')
            feat = line.split(', ')
            features.append(feat)
            
            # read next line
            line = f.readline()
        
        # create dataframe of features and append labels
        X = np.array(features, dtype = float, ndmin = 2)
        
        return X.transpose()

# 1) Linear Regression 

### Helper Functions 

In [92]:
# k = frequency increment
# d = function depth
# given input datapoint 'x_sample', 
# returns transformed version of the intput datapoint as a numpy array
def get_feature_vector(x_sample, k, d):
    
    # stored transformed values in a list
    trans_feat_list = []
    
    # append 1 and value of 'x_sample' to the list
    trans_feat_list.append(float(1))
    trans_feat_list.append(float(x_sample))
    
    # remaining transformations will be based on 'k' and 'd'
    for i in range(1, d+1):
        val1 = (np.sin(i*k*x_sample, dtype = float)**(i*k))*np.cos(x_sample, dtype = float)
        trans_feat_list.append(val1)
        val2 = (np.cos(i*k*x_sample, dtype = float)**(i*k))*np.sin(x_sample, dtype = float)
        trans_feat_list.append(val2)
    
    return np.array(trans_feat_list, dtype = float, ndmin = 2).transpose()


# calculates sample squared error
def get_sample_squared_error(y_sample, y_pred):
    
    # calculate and return squared error
    return np.square(y_sample - y_pred)


# get's prediction value for a sample
def get_prediction_value(x_sample, param_vec, k, d):
    
    # return prediction value
    return np.dot(param_vec, get_feature_vector(x_sample, k, d))


In [97]:
get_feature_vector(2, 1, 3).shape

(8, 1)

## Model Training & Prediction

In [30]:
# trains a linear regression model 
def train_model(X_train, Y_train, epochs, alpha, k, d):
    
    # get number of training samples
    n_feat, n_samples = X_train.shape
    
    # get output dimension
    n_out, __ = Y.train.shape
    
    # initialize parameter vector
    param_vec = np.random.randn(n_out, d+2)
    
    # do this per epoch
    for i in range(epochs):    
        for j in range(n_samples):
            
            # pick a sample randomly
            idx = np.random.randint(0, n_samples)
            x_sample = X_train[idx]
            y_sample = Y_train[idx]
            
            # get prediction value and adjust weights
            y_pred = get_prediction_value(x_sample, param_vec, k, d)
            gradient_vec = get_feature_vector(x_sample, k, g)*(y_pred - y_sample)
            
            # adjust parameter values using stochastic gradient descent
            param_vec = param_vec - (alpha*gradient_vec)
    
    # return final parameter vector
    return param_vec

# returns predicted values and squared error given test data
def get_prediction(X_test, Y_test, param_vec, k, d):
    
    # save number of test samples
    n_feat, n_test_samples = X_test.shape
    
    # initialize list to store squared error and predictions
    test_se = []
    predictions = []
    
    for i in range(n_test_samples):
        y_pred = get_prediction_value(X_test[i], params_vec, k, d)
        predictions.append(y_pred)
        sample_se = get_sample_squared_error(Y_test[i], y_pred)
        test_se.append(sample_se)
        
    return predictions, test_se

1

# Locally Weighted Linear Regression

# 3) Softmax Regression

### Helper Functions

#### one_hot_encoder(arr) : return encoded_arr, label_idx_dict
**arr:** <br>
[['Ceramic' 'Metal' 'Metal' 'Metal' 'Ceramic' 'Plastic' 'Plastic'
  'Plastic' 'Plastic' 'Plastic' 'Plastic' 'Ceramic']]<br>  
**encoded_arr:** <br>
[[0 1 1 1 0 0 0 0 0 0 0 0]<br>
 [0 0 0 0 0 1 1 1 1 1 1 0]<br>
 [1 0 0 0 1 0 0 0 0 0 0 1]] <br>
**label_idx_dict:** <br>
{'Metal': 0, 'Plastic': 1, 'Ceramic': 2}

In [90]:
# given an array of attribute values for a categocial attribute,
# preforms one-hot-encoding and returns resulting binary array
def one_hot_encoder(arr):
    
    __, n_samples = arr.shape
    
    # get unique labels
    uniq_labels = set(arr[0,:].tolist())
    
    # get number of total attribute values
    n_labels = len(uniq_labels)
    
    # create an array of size n_labels*n_samples to store encoded values
    encoded_arr = np.zeros((n_labels, n_samples), dtype = int)
    
    # create dictionary to store row indev of each attribute value
    label_idx_dict = {}
    for i, v in enumerate(uniq_labels):
        label_idx_dict[v] = i
        
    # fill encoded_arr using attribute index dictionary and input arr
    for i in range(n_samples):
        # get index to encode as 1
        idx = label_idx_dict[arr[0,i]]
        encoded_arr[idx, i] = 1
        
    return encoded_arr, label_idx_dict


#### reverse_encoder(arr_2D, label_idx_list) : return decoded_arr
**arr_2D:** <br>
[[0 1 1 1 0 0 0 0 0 0 0 0]<br>
 [0 0 0 0 0 1 1 1 1 1 1 0]<br>
 [1 0 0 0 1 0 0 0 0 0 0 1]] <br>
**label_idx_dict:** <br>
{'Metal': 0, 'Plastic': 1, 'Ceramic': 2} <br>
**idx_arr:** <br>
array([2, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 2]) <br>
**decoded_arr:** <br> 
[['Ceramic' 'Metal' 'Metal' 'Metal' 'Ceramic' 'Plastic' 'Plastic'
  'Plastic' 'Plastic' 'Plastic' 'Plastic' 'Ceramic']] <br>

In [91]:
# given an array of one hot encoded values and a dictionary with 'original attribute': encoded index info
# reverses the encoding and returns original attribute values
def reverse_encoder(arr_2D, label_idx_dict):
    
    # get no of samples and atttribute values
    n_labels, n_samples = arr_2D.shape
    
    # idx_arr gets and stores the index of the encoded '1' for each column/datapoint
    idx_arr = arr_2D * np.arange(n_labels, dtype = int).reshape(n_attrib,1)
    idx_arr = np.sum(idx_arr, axis = 0)
    
    # initialize list ot sore original labels
    orig_labels = []
    
    # get inverse of the dictionary
    inv_label_idx_dict = {v: k for k, v in label_idx_dict.items()}
    
    for i in range(n_samples):
        orig_labels.append(inv_label_idx_dict[idx_arr[i]])
    
    # return original labels as an array
    return np.array(orig_labels, dtype = str, ndmin = 2)


### Functions for Training and Prediction

In [106]:
# given a vector of parobaility values, returns label with max probability for a single sample
def get_sample_prediction_label(sfmax_net, label_idx_dict):
    
    # get inverse of the dictionary
    inv_label_idx_dict = {v: k for k, v in label_idx_dict.items()}
    
    # return label with max probability value
    return inv_label_idx_dict[np.argmax(sfmax_net, axis = 0)]


# uses softmax function and parameter matrix to get probability values
# for multiclass classification
def get_sample_prediction_values(x, param_mtx):
    
    # initialize column vector of ones to add bias
    x_sample = np.ones((x.shape[0]+1, 1), dtype = float)
    x_sample[1::] = x.reshape(x.shape[0], 1)
    
    # calculate linear net value
    net = np.dot(param_mtx, x_sample)
    
    # calculate exponential value for rach class
    exp_net = np.exp(net)
    
    # calculate softmax value for each class
    sfmax_net = exp_net/np.sum(exp_net, axis = 0)
    
    return sfmax_net


# trains a softmax regression model given training data, alpha and number of epochs
def train_softmax_regressor(X_train, Y_train, alpha, epochs):
    
    # get number of features and samples
    n_feat, n_samples = X_train.shape
    
    # get encoded array for y
    Y_train_encoded, label_idx_dict = one_hot_encoder(Y_train)
    
    # get no of classes/labels
    n_class, __ = Y_train_encoded.shape
    
    # get paramater matrix
    param_mtx = np.random.randn(n_class, n_feat+1)
    
    for i in range(epochs):
        for j in range(n_samples):
            
            # pick a sample randomly
            idx = np.random.randint(0, n_samples)
            x_sample = X_train[:,idx]
            y_sample = Y_train_encoded[:,idx].reshape(Y_train_encoded.shape[0], 1)
            
            # get prediction value and adjust weights
            y_pred = get_sample_prediction_values(x_sample, param_mtx)
            
            # calculate gradient matrix
            gradient_mtx = np.dot((y_sample - y_pred), x_sample.transpose())
            
            # adjust parameter values using stochastic gradient ascent
            param_mtx = param_mtx + alpha*gradient_mtx
            
    # return final parameter matrix
    return param_mtx, label_idx_dict

### Function for leave-one-out-evaluation

In [ ]:
def leave_one_out_evaluation(X_eval, Y_eval, alpha, epochs):
    
    # get number of features and samples
    n_feat, n_samples = X_eval.shape
    
    # prediction labels generated by 'predict_class_with_knn' will be stored in this list
    Y_pred = []
    
    for i in range(n_samples):
        
        # pick test datapoint
        x_test = X_train[:,i]
        y_test = Y_train[:,i]
        
        # create traiing set by deleting test datapoint
        X_train = np.delete(X_eval, i, axis = 1)
        Y_train = np.delete(Y_eval, i, axis = 1)
        
        # train model
        model_params, label_idx_dict = train_softmax_regressor(X_train, Y_train, alpha, epochs)
        
        # get test data prediction
        y_pred_values = get_sample_prediction_values(x_test, model_params)
        y_pred_label = get_sample_prediction_label(y_pred_values, label_idx_list)
        Y_pred.append(y_pred_label)
    
    # convert prediction list to numpy array
    Y_pred = np.array(Y_pred, dtype = str, ndmin = 2)
    
    # return accuracy
    return (np.sum(Y_eval == Y_pred))/n_samples


### Provide Filenames:
1) Training/evaluation file: name of a .txt file with each line containing training/evaluation features(x) and label(y) in the following format:
((x1, x2, .....xn), y)

2) Test file: name of a .txt file with each line containing test features(x) in the following format:
(x1, x2, .....xn)

In [48]:
'''
fname_train = str(input('Enter file containing training data: '))
fname_test = str(input('Enter file containing test data: '))
fname_eval = str(input('Enter file containing leave_one_out evaluation data: '))
'''
fname_train = '3_train.txt'
fname_test = '3_test.txt'
fname_eval = '3_eval.txt'

X_train, Y_train = get_X_Y_arrays(fname_train)
X_test = get_X_array(fname_test)
X_eval, Y_eval = get_X_Y_arrays(fname_eval)

In [88]:
Y_train

array([['Ceramic', 'Metal', 'Metal', 'Metal', 'Ceramic', 'Plastic',
        'Plastic', 'Plastic', 'Plastic', 'Plastic', 'Plastic', 'Ceramic']],
      dtype='<U7')

In [77]:
arr, my_dict = one_hot_encoder(Y_train)

In [89]:
reverse_encoder(arr, my_dict)

array([['Ceramic', 'Metal', 'Metal', 'Metal', 'Ceramic', 'Plastic',
        'Plastic', 'Plastic', 'Plastic', 'Plastic', 'Plastic', 'Ceramic']],
      dtype='<U7')

In [107]:
np.argmax(np.array([[0.2],[1.5],[1]]))

1

In [101]:
# calculate exponential value for rach class
exp_net = np.exp(np.array([[0.2],[1.5],[1]]))

# calculate softmax value for each class
exp_net/np.sum(exp_net, axis = 0)

array([[0.14503605],
       [0.53218029],
       [0.32278366]])

In [105]:
np.ones((5+1, 1), dtype = float).shape

(6, 1)